<a href="https://colab.research.google.com/github/mohd-muzamil/ML_DL/blob/main/boilerplate_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Script contains the broiler code to train Deep Learning models using Pytorch

In [15]:
#Imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

 #### Steps
1. download train-test data
1. create dataloaders
1. create Models
1. train-test methods
1. model training
1. model testing and model evaluation
1. model save

In [16]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [17]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("Train Data: ", training_data)
print(f"\nShape of train dataset: [batch_size:{batch_size}, No of batches:{len(train_dataloader)}]")
print("\nTest Data: ", test_data)
print(f"\nShape of test dataset: [batch_size:{batch_size}, No of batches:{len(test_dataloader)}\n")

print("Data in a Single batch:")
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Train Data:  Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

Shape of train dataset: [batch_size:64, No of batches:938]

Test Data:  Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

Shape of test dataset: [batch_size:64, No of batches:157

Data in a Single batch:
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [18]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
# training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295050  [    0/60000]
loss: 2.277929  [ 6400/60000]
loss: 2.262930  [12800/60000]
loss: 2.264566  [19200/60000]
loss: 2.240314  [25600/60000]
loss: 2.221179  [32000/60000]
loss: 2.230494  [38400/60000]
loss: 2.196205  [44800/60000]
loss: 2.194537  [51200/60000]
loss: 2.168874  [57600/60000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 2.151086 

Epoch 2
-------------------------------
loss: 2.158328  [    0/60000]
loss: 2.141770  [ 6400/60000]
loss: 2.089203  [12800/60000]
loss: 2.111807  [19200/60000]
loss: 2.055007  [25600/60000]
loss: 2.002734  [32000/60000]
loss: 2.033614  [38400/60000]
loss: 1.951588  [44800/60000]
loss: 1.960531  [51200/60000]
loss: 1.893380  [57600/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.878567 

Epoch 3
-------------------------------
loss: 1.907865  [    0/60000]
loss: 1.872892  [ 6400/60000]
loss: 1.761781  [12800/60000]
loss: 1.806778  [19200/60000]
loss: 1.696354  [25600/60000]
loss: 1.647940  [32000/600

In [23]:
# model save
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [24]:
# model evaluate
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
